In [1]:
import torch

# GPU 사용 가능 여부
print(torch.cuda.is_available())  # True면 GPU 사용 가능
print(torch.cuda.get_device_name(0))  # GPU 이름 출력

True
Tesla T4


In [2]:
# Google Colab 환경을 위한 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# === One-Cell Clean Preprocessing (공휴일여부 + 휴무일여부) ===
import pandas as pd
import numpy as np

TRAIN_PATH = '/content/drive/MyDrive/lg_aimers_2/data/train/train.csv'
OUT_PATH   = '/content/drive/MyDrive/lg_aimers_2/train_preprocessed_05.csv'

try:
    import holidays
except ModuleNotFoundError:
    !pip install holidays -q
    import holidays


def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """train.csv 전처리 + (공휴일/휴무일) + 계절/사이클릭 + 출시일 파생"""
    df = df.copy()

     # === 음수 매출수량 보정 ===
    if "매출수량" in df.columns:
        df["매출수량"] = df["매출수량"].clip(lower=0)

    df['영업일자'] = pd.to_datetime(df['영업일자'])
    df['년'] = df['영업일자'].dt.year
    df['월'] = df['영업일자'].dt.month
    df['일'] = df['영업일자'].dt.day
    df['요일'] = df['영업일자'].dt.dayofweek
    df['주말여부'] = df['요일'].isin([5, 6])

    # 메뉴 분리
    df[['영업장명', '메뉴명']] = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)

    # === 캘린더 피처 ===
    def add_calendar_features(frame: pd.DataFrame, date_col="영업일자") -> pd.DataFrame:
        out = frame.copy()
        d = pd.to_datetime(out[date_col])
        years = sorted(d.dt.year.unique())
        try:
            KR_HOL = holidays.KR(years=years, language="ko")
        except Exception:
            KR_HOL = holidays.KR(years=years)

        # 공휴일 여부만
        out["공휴일여부"] = d.dt.date.map(lambda x: x in KR_HOL)

        # 계절
        m = d.dt.month
        out["계절(겨울0봄1여름2가을3)"] = (
            (m.isin([12,1,2]))*0 +
            (m.isin([3,4,5]))*1 +
            (m.isin([6,7,8]))*2 +
            (m.isin([9,10,11]))*3
        ).astype("int8")

        # 사이클릭
        out["요일_sin"] = np.sin(2*np.pi*out["요일"]/7)
        out["요일_cos"] = np.cos(2*np.pi*out["요일"]/7)
        out["월_sin"]   = np.sin(2*np.pi*(out["월"]-1)/12)
        out["월_cos"]   = np.cos(2*np.pi*(out["월"]-1)/12)

        return out

    df = add_calendar_features(df, date_col='영업일자')

    # === 추가: 휴무일여부 (주말 OR 공휴일) ===
    df["휴무일여부"] = df["주말여부"] | df["공휴일여부"]

    # === 출시일 파생 ===
    launch_dates = {
        '느티나무 셀프BBQ_1인 수저세트': '2023-01-17', '느티나무 셀프BBQ_BBQ55(단체)': '2023-01-05',
        '느티나무 셀프BBQ_대여료 90,000원': '2023-01-02', '느티나무 셀프BBQ_본삼겹 (단품,실내)': '2023-01-03',
        '느티나무 셀프BBQ_스프라이트 (단체)': '2023-01-03', '느티나무 셀프BBQ_신라면': '2023-04-14',
        '느티나무 셀프BBQ_쌈야채세트': '2023-01-11', '느티나무 셀프BBQ_쌈장': '2023-04-14',
        '느티나무 셀프BBQ_육개장 사발면': '2023-04-14', '느티나무 셀프BBQ_일회용 소주컵': '2023-01-23',
        '느티나무 셀프BBQ_일회용 종이컵': '2023-01-22', '느티나무 셀프BBQ_잔디그늘집 대여료 (12인석)': '2023-04-14',
        '느티나무 셀프BBQ_잔디그늘집 대여료 (6인석)': '2023-01-05', '느티나무 셀프BBQ_잔디그늘집 의자 추가': '2023-04-14',
        '느티나무 셀프BBQ_참이슬 (단체)': '2023-01-03', '느티나무 셀프BBQ_친환경 접시 14cm': '2023-01-22',
        '느티나무 셀프BBQ_친환경 접시 23cm': '2023-01-05', '느티나무 셀프BBQ_카스 병(단체)': '2023-01-03',
        '느티나무 셀프BBQ_콜라 (단체)': '2023-01-03', '느티나무 셀프BBQ_햇반': '2023-04-14',
        '느티나무 셀프BBQ_허브솔트': '2023-04-14', '담하_(단체) 공깃밥': '2023-03-13',
        '담하_(단체) 생목살 김치전골 2.0': '2023-09-18', '담하_(단체) 은이버섯 갈비탕': '2023-06-12',
        '담하_(단체) 한우 우거지 국밥': '2023-01-06', '담하_(단체) 황태해장국 3/27까지': '2023-01-07',
        '담하_(정식) 된장찌개': '2023-06-03', '담하_(정식) 물냉면 ': '2023-06-03',
        '담하_(정식) 비빔냉면': '2023-06-03', '담하_(후식) 물냉면': '2023-06-02',
        '담하_(후식) 비빔냉면': '2023-06-02', '담하_갑오징어 비빔밥': '2023-03-17',
        '담하_갱시기': '2023-12-08', '담하_꼬막 비빔밥': '2023-09-08',
        '담하_느린마을 막걸리': '2023-01-02', '담하_담하 한우 불고기 정식': '2023-06-02',
        '담하_더덕 한우 지짐': '2023-09-09', '담하_라면사리': '2023-01-04',
        '담하_룸 이용료': '2023-01-03', '담하_명인안동소주': '2023-07-01',
        '담하_명태회 비빔냉면': '2023-06-02', '담하_문막 복분자 칵테일': '2023-09-12',
        '담하_봉평메밀 물냉면': '2023-06-02', '담하_제로콜라': '2023-01-05',
        '담하_처음처럼': '2023-01-03', '담하_하동 매실 칵테일': '2023-03-18',
        '라그로타_AUS (200g)': '2023-12-08', '라그로타_G-Charge(3)': '2023-01-02',
        '라그로타_Open Food': '2023-01-07', '라그로타_그릴드 비프 샐러드': '2023-09-08',
        '라그로타_까르보나라': '2023-12-08', '라그로타_모둠 해산물 플래터': '2023-09-09',
        '라그로타_미션 서드 카베르네 쉬라': '2023-01-02', '라그로타_버섯 크림 리조또': '2023-12-08',
        '라그로타_시저 샐러드 ': '2023-09-08', '라그로타_알리오 에 올리오 ': '2023-09-08',
        '라그로타_양갈비 (4ps)': '2023-09-10', '라그로타_한우 (200g)': '2023-12-09',
        '라그로타_해산물 토마토 스튜 파스타': '2023-12-08', '미라시아_(단체)브런치주중 36,000': '2023-01-03',
        '미라시아_(오븐) 하와이안 쉬림프 피자': '2023-09-09', '미라시아_BBQ 고기추가': '2023-01-05',
        '미라시아_글라스와인 (레드)': '2023-01-02', '미라시아_레인보우칵테일(알코올)': '2023-01-02',
        '미라시아_버드와이저(무제한)': '2023-04-21', '미라시아_보일링 랍스타 플래터': '2023-06-05',
        '미라시아_보일링 랍스타 플래터(덜매운맛)': '2023-06-03', '미라시아_브런치(대인) 주중': '2023-01-02',
        '미라시아_쉬림프 투움바 파스타': '2023-06-03', '미라시아_스텔라(무제한)': '2023-04-21',
        '미라시아_스프라이트': '2023-06-02', '미라시아_얼그레이 하이볼': '2023-01-02',
        '미라시아_유자 하이볼': '2023-03-17', '미라시아_잭 애플 토닉': '2023-09-09',
        '미라시아_칠리 치즈 프라이': '2023-06-03', '미라시아_코카콜라': '2023-06-02',
        '미라시아_코카콜라(제로)': '2023-06-12', '미라시아_콥 샐러드': '2023-12-08',
        '미라시아_파스타면 추가(150g)': '2023-06-03', '미라시아_핑크레몬에이드': '2023-03-17',
        '연회장_Cass Beer': '2023-01-06', '연회장_Conference L1': '2023-01-03',
        '연회장_Conference L2': '2023-01-11', '연회장_Conference L3': '2023-01-05',
        '연회장_Conference M1': '2023-01-06', '연회장_Conference M8': '2023-01-09',
        '연회장_Conference M9': '2023-01-06', '연회장_Convention Hall': '2023-01-03',
        '연회장_Cookie Platter': '2023-01-09', '연회장_Grand Ballroom': '2023-01-06',
        '연회장_OPUS 2': '2023-01-05', '연회장_Regular Coffee': '2023-02-24',
        '연회장_공깃밥': '2023-07-21', '연회장_마라샹궈': '2023-09-08',
        '연회장_매콤 무뼈닭발&계란찜': '2023-01-02', '연회장_삼겹살추가 (200g)': '2023-07-21',
        '연회장_왕갈비치킨': '2023-07-22', '카페테리아_단체식 13000(신)': '2023-04-18',
        '카페테리아_단체식 18000(신)': '2023-04-05', '카페테리아_진사골 설렁탕': '2023-12-06',
        '카페테리아_한상 삼겹구이 정식(2인) 소요시간 약 15~20분': '2023-03-17',
        '화담숲주막_느린마을 막걸리': '2023-03-31', '화담숲주막_단호박 식혜 ': '2023-03-31',
        '화담숲주막_병천순대': '2023-03-31', '화담숲주막_스프라이트': '2023-03-31',
        '화담숲주막_참살이 막걸리': '2023-03-31', '화담숲주막_찹쌀식혜': '2023-03-31',
        '화담숲주막_콜라': '2023-03-31', '화담숲주막_해물파전': '2023-03-31',
        '화담숲카페_메밀미숫가루': '2023-03-31', '화담숲카페_아메리카노 HOT': '2023-03-31',
        '화담숲카페_아메리카노 ICE': '2023-03-31', '화담숲카페_카페라떼 ICE': '2023-03-31',
        '화담숲카페_현미뻥스크림': '2023-03-31'
    }
    launch_dates = {k: pd.to_datetime(v) for k, v in launch_dates.items()}

    def calculate_days_since_launch(row):
        menu = row['영업장명_메뉴명']
        if menu in launch_dates:
            launch_date = launch_dates[menu]
            if row['영업일자'] >= launch_date:
                return (row['영업일자'] - launch_date).days
        return 0

    df['출시일로부터경과일'] = df.apply(calculate_days_since_launch, axis=1)

    return df


# 실행
train_df = pd.read_csv(TRAIN_PATH)
train_preprocessed = preprocess_data(train_df)
train_preprocessed.to_csv(OUT_PATH, index=False, encoding='utf-8-sig')
print("전처리 완료 →", OUT_PATH)

# 점검
print("공휴일여부 분포:", train_preprocessed["공휴일여부"].value_counts(dropna=False).to_dict())
print("휴무일여부 분포:", train_preprocessed["휴무일여부"].value_counts(dropna=False).to_dict())


전처리 완료 → /content/drive/MyDrive/lg_aimers_2/train_preprocessed_05.csv
공휴일여부 분포: {False: 97079, True: 5597}
휴무일여부 분포: {False: 69287, True: 33389}


In [4]:
# ===== Colab 셋업 =====
!pip -q install optuna lightgbm

import os, joblib, optuna, numpy as np, pandas as pd
from lightgbm import LGBMRegressor, log_evaluation, early_stopping

# -----------------------------
# 0) 경로/상수
# -----------------------------
TRAIN_PATH = "/content/drive/MyDrive/lg_aimers_2/train_preprocessed_05.csv"
MODEL_OUT  = "/content/drive/MyDrive/lg_aimers_2/models/lgbm_05_optuna.pkl"
USE_GPU    = True
DEVICE     = {"device": "gpu"} if USE_GPU else {"device": "cpu"}
MAX_LAG    = 28  # 너의 lag 최댓값과 동일해야 함

# -----------------------------
# 1) 데이터 로드 & 기본 세팅
# -----------------------------
df = pd.read_csv(TRAIN_PATH, parse_dates=["영업일자"])

# 음수 매출 안전 처리(대회 데이터 이슈 대응)
df["매출수량"] = pd.to_numeric(df["매출수량"], errors="coerce").fillna(0)
df["매출수량"] = df["매출수량"].clip(lower=0)

# bool -> int
for c in [c for c in ["주말여부","공휴일여부","휴무일여부","신규메뉴여부"] if c in df.columns]:
    df[c] = df[c].astype(int)

# key 컬럼
key_col = "영업장명_메뉴명"
df[key_col] = df[key_col].astype("category")

# 시간 정렬
df = df.sort_values([key_col, "영업일자"]).copy()

# -----------------------------
# 2) lag/rolling/same_dow 생성 (네 기존 규칙 유지)
# -----------------------------
lag_list = [1, 7, 14, 28]
for lag in lag_list:
    df[f"lag_{lag}"] = df.groupby(key_col)["매출수량"].shift(lag)

def add_rolling(g, window, how="mean"):
    s = g["매출수량"].shift(1)  # 과거만
    if how == "mean":
        return s.rolling(window, min_periods=1).mean()
    elif how == "sum":
        return s.rolling(window, min_periods=1).sum()

df["roll7_mean"]  = df.groupby(key_col, group_keys=False).apply(add_rolling, window=7,  how="mean")
df["roll7_sum"]   = df.groupby(key_col, group_keys=False).apply(add_rolling, window=7,  how="sum")
df["roll14_mean"] = df.groupby(key_col, group_keys=False).apply(add_rolling, window=14, how="mean")
df["roll28_mean"] = df.groupby(key_col, group_keys=False).apply(add_rolling, window=28, how="mean")

def same_dow_mean_28(g):
    out = np.full(len(g), np.nan, dtype=float)
    vals = g["매출수량"].shift(1)
    dows = g["요일"]
    for i in range(len(g)):
        lo = max(0, i-28)
        same_idx = [j for j in range(lo, i) if dows.iloc[j] == dows.iloc[i]]
        if same_idx:
            out[i] = vals.iloc[same_idx].mean()
    return pd.Series(out, index=g.index)

if "요일" in df.columns:
    df["same_dow_mean_28"] = df.groupby(key_col, group_keys=False).apply(same_dow_mean_28)

# 신규메뉴 파생(있을 때만)
if "출시일로부터경과일" in df.columns:
    df["출시후_주차"]   = (df["출시일로부터경과일"] // 7).clip(lower=0).astype(int)
    df["출시_0주"]     = (df["출시후_주차"] == 0).astype(int)
    df["출시_1_2주"]   = df["출시후_주차"].between(1,2).astype(int)
    df["출시_3_4주"]   = df["출시후_주차"].between(3,4).astype(int)
    df["출시_5주이상"] = (df["출시후_주차"] >= 5).astype(int)

# -----------------------------
# 3) 학습에 쓸 피처 목록 (네 기준 유지 + 휴무일여부 포함)
# -----------------------------
base_feats = [c for c in [
    "년","월","일","요일","주말여부","공휴일여부","휴무일여부","신규메뉴여부",
    "출시일로부터경과일","출시후_주차","출시_0주","출시_1_2주","출시_3_4주","출시_5주이상",
    "요일_sin","요일_cos","월_sin","월_cos","계절(겨울0봄1여름2가을3)"
] if c in df.columns]

lag_feats  = [f"lag_{l}" for l in lag_list]
roll_feats = [c for c in ["roll7_mean","roll7_sum","roll14_mean","roll28_mean","same_dow_mean_28"] if c in df.columns]

use_cols = base_feats + lag_feats + roll_feats + [key_col]
cat_features = [use_cols.index(key_col)]  # LightGBM용 범주형 인덱스

# -----------------------------
# 4) 폴드 정의(롤링) + gap purge
#    - 아래는 예시: 14일 검증창을 여러 번 슬라이딩
#    - 네 데이터의 마지막: 2024-06-15 기준으로 몇 개 컷 생성
# -----------------------------
def build_folds(df, val_window_days=14, n_folds=4, last_val_end="2024-06-15"):
    last_val_end = pd.Timestamp(last_val_end)
    folds = []
    for k in range(n_folds):
        val_end   = last_val_end - pd.Timedelta(days=(n_folds-1-k)*val_window_days)
        val_start = val_end - pd.Timedelta(days=val_window_days-1)
        # train은 val_start - MAX_LAG 까지만 사용 (누수 방지)
        train_end = val_start - pd.Timedelta(days=MAX_LAG)
        folds.append((train_end, val_start, val_end))
    return folds

folds = build_folds(df, val_window_days=14, n_folds=4, last_val_end="2024-06-15")
print("Folds:")
for i,(te,vs,ve) in enumerate(folds,1):
    print(f"Fold{i}: train_end={te.date()} | val={vs.date()}~{ve.date()}")

# -----------------------------
# 5) 폴드 데이터 구성 함수
# -----------------------------
def make_fold_data(df, use_cols, key_col, tr_end, va_st, va_en):
    tr = df[df["영업일자"] <= tr_end].copy()
    va = df[(df["영업일자"] >= va_st) & (df["영업일자"] <= va_en)].copy()

    # 최소 lag 확보(가장 작은 lag만 체크)
    tr = tr.dropna(subset=["lag_1"])
    va = va.dropna(subset=["lag_1"])

    # NA 대체
    fill_cols = list(set(use_cols) - {key_col})
    tr[fill_cols] = tr[fill_cols].fillna(0)
    va[fill_cols] = va[fill_cols].fillna(0)

    X_tr, y_tr = tr[use_cols], tr["매출수량"].astype(float).values
    X_va, y_va = va[use_cols], va["매출수량"].astype(float).values
    return X_tr, y_tr, X_va, y_va

# -----------------------------
# 6) 폴드 평균 RMSE 계산
# -----------------------------
def cv_rmse(params, df, use_cols, key_col, cat_features, nonneg_target=False):
    rmses = []
    for (tr_end, va_st, va_en) in folds:
        # gap purge: train_end를 val_start - MAX_LAG 로 강제
        te = va_st - pd.Timedelta(days=MAX_LAG)
        X_tr, y_tr, X_va, y_va = make_fold_data(df, use_cols, key_col, te, va_st, va_en)

        # poisson/tweedie 평가를 원하면 비음수 타깃 사용
        if nonneg_target:
            y_tr = np.clip(y_tr, 0, None)
            y_va = np.clip(y_va, 0, None)

        model = LGBMRegressor(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="rmse",
            categorical_feature=cat_features,
            callbacks=[log_evaluation(200), early_stopping(200)],
        )
        pred = model.predict(X_va, num_iteration=getattr(model, "best_iteration_", None))
        rmse = float(np.sqrt(((y_va - pred)**2).mean()))
        rmses.append(rmse)
    return float(np.mean(rmses))

# -----------------------------
# 7) Optuna 목적함수 (시계열 CV + gap)
# -----------------------------
def objective(trial):
    obj = trial.suggest_categorical("objective", ["regression","poisson","tweedie"])
    params = dict(
        objective=obj,
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.05, log=True),
        n_estimators=trial.suggest_int("n_estimators", 3000, 9000, step=1000),
        num_leaves=trial.suggest_int("num_leaves", 63, 191, step=16),
        min_child_samples=trial.suggest_int("min_child_samples", 80, 220, step=20),
        subsample=trial.suggest_float("subsample", 0.75, 0.95),
        subsample_freq=1,
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.6, 0.9),
        reg_alpha=trial.suggest_float("reg_alpha", 0.0, 0.6),
        reg_lambda=trial.suggest_float("reg_lambda", 0.5, 4.0),
        min_split_gain=trial.suggest_float("min_split_gain", 0.0, 0.2),
        random_state=42, n_jobs=-1, **DEVICE
    )
    if obj == "tweedie":
        params["tweedie_variance_power"] = trial.suggest_float("tweedie_variance_power", 1.1, 1.6)

    # poisson/tweedie는 비음수 타깃으로 CV
    nonneg = (obj in {"poisson","tweedie"})
    score = cv_rmse(params, df, use_cols, key_col, cat_features, nonneg_target=nonneg)
    return score

study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best RMSE :", study.best_value)
print("Best Params:", study.best_params)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.1 MB/s eta 0:00:00


/tmp/ipython-input-2228773888.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"lag_{lag}"] = df.groupby(key_col)["매출수량"].shift(lag)
/tmp/ipython-input-2228773888.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"lag_{lag}"] = df.groupby(key_col)["매출수량"].shift(lag)
/tmp/ipython-input-2228773888.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"lag_{lag}"] = df.groupby(key_col)["매출수량"].shift(lag)
/

Folds:
Fold1: train_end=2024-03-24 | val=2024-04-21~2024-05-04
Fold2: train_end=2024-04-07 | val=2024-05-05~2024-05-18
Fold3: train_end=2024-04-21 | val=2024-05-19~2024-06-01
Fold4: train_end=2024-05-05 | val=2024-06-02~2024-06-15


  0%|          | 0/50 [00:00<?, ?it/s]

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2915
[LightGBM] [Info] Number of data points in the train set: 86464, number of used features: 28
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 21 dense feature groups (1.98 MB) transferred to GPU in 0.005085 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.397748
Training until validation scores don't improve for 200 rounds
[200]	valid_0's rmse: 22.5561	valid_0's poisson: -27.5713
[400]	valid_0's rmse: 23.5216	valid_0's poisson: -27.5219
Early stopping, best iteration is:
[242]	valid_0's rmse: 22.415	valid_0's poisson: -27.748
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2917
[LightGBM] [Info] Number of data points in the train set: 89166, number of used features

In [5]:
import json, os

BEST_PARAMS_PATH = "/content/drive/MyDrive/lg_aimers_2/models/lgbm_05_best_params.json"

best_params = study.best_params.copy()
with open(BEST_PARAMS_PATH, "w") as f:
    json.dump(best_params, f, indent=2, ensure_ascii=False)

print("✅ saved best params ->", BEST_PARAMS_PATH)

import joblib
META_PATH = "/content/drive/MyDrive/lg_aimers_2/models/lgbm_05_meta.pkl"
joblib.dump({"use_cols": use_cols, "cat_idx": cat_features}, META_PATH)
print("✅ saved meta ->", META_PATH)


✅ saved best params -> /content/drive/MyDrive/lg_aimers_2/models/lgbm_05_best_params.json
✅ saved meta -> /content/drive/MyDrive/lg_aimers_2/models/lgbm_05_meta.pkl


In [6]:
import json, joblib
from lightgbm import LGBMRegressor, log_evaluation, early_stopping

BEST_PARAMS_PATH = "/content/drive/MyDrive/lg_aimers_2/models/lgbm_05_best_params.json"
META_PATH        = "/content/drive/MyDrive/lg_aimers_2/models/lgbm_05_meta.pkl"
MODEL_OUT        = "/content/drive/MyDrive/lg_aimers_2/models/lgbm_05.pkl"

# 1) 파라미터 로드
with open(BEST_PARAMS_PATH, "r") as f:
    best_params = json.load(f)

# Tweedie가 아니라면 안전하게 제거
if best_params.get("objective") != "tweedie":
    best_params.pop("tweedie_variance_power", None)

# GPU/공통 설정 부착
USE_GPU = True
best_params.update({"device": "gpu" if USE_GPU else "cpu", "random_state": 42, "n_jobs": -1})

# 2) 메타 로드(use_cols, cat_idx) — 없으면 스크립트에서 재생성해도 OK
meta = joblib.load(META_PATH)
use_cols = meta["use_cols"]
cat_features = meta["cat_idx"]  # [index]
key_col = use_cols[cat_features[0]]

# 3) 홀드아웃 + early_stopping으로 best_iteration_
VALID_START = pd.Timestamp("2024-06-01")
VALID_END   = pd.Timestamp("2024-06-15")
MAX_LAG     = 28

train_end = VALID_START - pd.Timedelta(days=MAX_LAG)
df_tr = df[df["영업일자"] <= train_end].copy()
df_va = df[(df["영업일자"] >= VALID_START) & (df["영업일자"] <= VALID_END)].copy()

df_tr = df_tr.dropna(subset=["lag_1"])
df_va = df_va.dropna(subset=["lag_1"])
fill_cols = list(set(use_cols) - {key_col})
df_tr[fill_cols] = df_tr[fill_cols].fillna(0)
df_va[fill_cols] = df_va[fill_cols].fillna(0)

X_tr, y_tr = df_tr[use_cols], df_tr["매출수량"].astype(float).values
X_va, y_va = df_va[use_cols], df_va["매출수량"].astype(float).values

if best_params["objective"] in {"poisson","tweedie"}:
    y_tr = np.clip(y_tr, 0, None)
    y_va = np.clip(y_va, 0, None)

finder_params = best_params.copy()
finder_params["n_estimators"] = 20000  # 크게 주고 early_stopping으로 최적 라운드 탐색
finder = LGBMRegressor(**finder_params)
finder.fit(
    X_tr, y_tr,
    eval_set=[(X_va, y_va)],
    eval_metric="rmse",
    categorical_feature=cat_features,
    callbacks=[log_evaluation(200), early_stopping(800)],
)
best_iter = int(finder.best_iteration_)
print("best_iteration_:", best_iter)

# 4) 전체 데이터로 refit
df_final = df.dropna(subset=["lag_1"]).copy()
df_final[fill_cols] = df_final[fill_cols].fillna(0)
X_final = df_final[use_cols]
y_final = df_final["매출수량"].astype(float).values
if best_params["objective"] in {"poisson","tweedie"}:
    y_final = np.clip(y_final, 0, None)

refit_params = best_params.copy()
refit_params["n_estimators"] = best_iter
final_model = LGBMRegressor(**refit_params)
final_model.fit(
    X_final, y_final,
    categorical_feature=cat_features,
)

joblib.dump({"model": final_model, "features": list(X_final.columns), "cat_idx": cat_features}, MODEL_OUT)
print("✅ Saved ->", MODEL_OUT)


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2921
[LightGBM] [Info] Number of data points in the train set: 94377, number of used features: 28
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 21 dense feature groups (2.16 MB) transferred to GPU in 0.008248 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.391786
Training until validation scores don't improve for 800 rounds
[200]	valid_0's rmse: 15.244	valid_0's tweedie: 16.7256
[400]	valid_0's rmse: 15.6147	valid_0's tweedie: 16.9915
[600]	valid_0's rmse: 15.659	valid_0's tweedie: 17.0967
[800]	valid_0's rmse: 16.0053	valid_0's tweedie: 17.323
Early stopping, best iteration is:
[167]	valid_0's rmse: 14.9939	valid_0's tweedie: 16.6204
best_iteration_: 167
[LightGBM] [Info] This is the GPU t

In [7]:
# !pip install lightgbm -q
import os, glob, joblib
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor

# ===== 경로 설정 =====
MODEL_PATH   = "/content/drive/MyDrive/lg_aimers_2/models/lgbm_05.pkl"
TEST_DIR     = "/content/drive/MyDrive/lg_aimers_2/data/test"   # TEST_00.csv ~ TEST_09.csv
SAMPLE_PATH  = "/content/drive/MyDrive/lg_aimers_2/data/sample_submission.csv"  # 필요 시 수정
OUT_PATH     = "/content/drive/MyDrive/lg_aimers_2/submission_lightgbm_05.csv"

# ===== 모델 로드 =====
bundle = joblib.load(MODEL_PATH)
model: LGBMRegressor = bundle["model"]
use_cols = bundle["features"]
cat_idx  = bundle["cat_idx"]
cat_col  = use_cols[cat_idx[0]]

# ===== 공휴일 계산: holidays 라이브러리 사용 =====
try:
    import holidays
except ModuleNotFoundError:
    # Colab 등에서 미설치 시
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "holidays", "-q"], check=True)
    import holidays

# 연도별 캐시(테스트 파일마다 연도가 다를 수 있으므로 지연 생성)
_HOL_CACHE = {}
def is_holiday(ts: pd.Timestamp) -> bool:
    y = int(ts.year)
    if y not in _HOL_CACHE:
        try:
            _HOL_CACHE[y] = holidays.KR(years=[y], language="ko")
        except Exception:
            _HOL_CACHE[y] = holidays.KR(years=[y])
    return ts.date() in _HOL_CACHE[y]

# ===== 출시 정보 (전처리와 동일 규칙 사용: dict 기반) =====
launch_dates = {
    '느티나무 셀프BBQ_1인 수저세트': '2023-01-17', '느티나무 셀프BBQ_BBQ55(단체)': '2023-01-05',
    '느티나무 셀프BBQ_대여료 90,000원': '2023-01-02', '느티나무 셀프BBQ_본삼겹 (단품,실내)': '2023-01-03',
    '느티나무 셀프BBQ_스프라이트 (단체)': '2023-01-03', '느티나무 셀프BBQ_신라면': '2023-04-14',
    '느티나무 셀프BBQ_쌈야채세트': '2023-01-11', '느티나무 셀프BBQ_쌈장': '2023-04-14',
    '느티나무 셀프BBQ_육개장 사발면': '2023-04-14', '느티나무 셀프BBQ_일회용 소주컵': '2023-01-23',
    '느티나무 셀프BBQ_일회용 종이컵': '2023-01-22', '느티나무 셀프BBQ_잔디그늘집 대여료 (12인석)': '2023-04-14',
    '느티나무 셀프BBQ_잔디그늘집 대여료 (6인석)': '2023-01-05', '느티나무 셀프BBQ_잔디그늘집 의자 추가': '2023-04-14',
    '느티나무 셀프BBQ_참이슬 (단체)': '2023-01-03', '느티나무 셀프BBQ_친환경 접시 14cm': '2023-01-22',
    '느티나무 셀프BBQ_친환경 접시 23cm': '2023-01-05', '느티나무 셀프BBQ_카스 병(단체)': '2023-01-03',
    '느티나무 셀프BBQ_콜라 (단체)': '2023-01-03', '느티나무 셀프BBQ_햇반': '2023-04-14',
    '느티나무 셀프BBQ_허브솔트': '2023-04-14', '담하_(단체) 공깃밥': '2023-03-13',
    '담하_(단체) 생목살 김치전골 2.0': '2023-09-18', '담하_(단체) 은이버섯 갈비탕': '2023-06-12',
    '담하_(단체) 한우 우거지 국밥': '2023-01-06', '담하_(단체) 황태해장국 3/27까지': '2023-01-07',
    '담하_(정식) 된장찌개': '2023-06-03', '담하_(정식) 물냉면 ': '2023-06-03',
    '담하_(정식) 비빔냉면': '2023-06-03', '담하_(후식) 물냉면': '2023-06-02',
    '담하_(후식) 비빔냉면': '2023-06-02', '담하_갑오징어 비빔밥': '2023-03-17',
    '담하_갱시기': '2023-12-08', '담하_꼬막 비빔밥': '2023-09-08',
    '담하_느린마을 막걸리': '2023-01-02', '담하_담하 한우 불고기 정식': '2023-06-02',
    '담하_더덕 한우 지짐': '2023-09-09', '담하_라면사리': '2023-01-04',
    '담하_룸 이용료': '2023-01-03', '담하_명인안동소주': '2023-07-01',
    '담하_명태회 비빔냉면': '2023-06-02', '담하_문막 복분자 칵테일': '2023-09-12',
    '담하_봉평메밀 물냉면': '2023-06-02', '담하_제로콜라': '2023-01-05',
    '담하_처음처럼': '2023-01-03', '담하_하동 매실 칵테일': '2023-03-18',
    '라그로타_AUS (200g)': '2023-12-08', '라그로타_G-Charge(3)': '2023-01-02',
    '라그로타_Open Food': '2023-01-07', '라그로타_그릴드 비프 샐러드': '2023-09-08',
    '라그로타_까르보나라': '2023-12-08', '라그로타_모둠 해산물 플래터': '2023-09-09',
    '라그로타_미션 서드 카베르네 쉬라': '2023-01-02', '라그로타_버섯 크림 리조또': '2023-12-08',
    '라그로타_시저 샐러드 ': '2023-09-08', '라그로타_알리오 에 올리오 ': '2023-09-08',
    '라그로타_양갈비 (4ps)': '2023-09-10', '라그로타_한우 (200g)': '2023-12-09',
    '라그로타_해산물 토마토 스튜 파스타': '2023-12-08',
    '미라시아_(단체)브런치주중 36,000': '2023-01-03',
    '미라시아_(오븐) 하와이안 쉬림프 피자': '2023-09-09', '미라시아_BBQ 고기추가': '2023-01-05',
    '미라시아_글라스와인 (레드)': '2023-01-02', '미라시아_레인보우칵테일(알코올)': '2023-01-02',
    '미라시아_버드와이저(무제한)': '2023-04-21', '미라시아_보일링 랍스타 플래터': '2023-06-05',
    '미라시아_보일링 랍스타 플래터(덜매운맛)': '2023-06-03', '미라시아_브런치(대인) 주중': '2023-01-02',
    '미라시아_쉬림프 투움바 파스타': '2023-06-03', '미라시아_스텔라(무제한)': '2023-04-21',
    '미라시아_스프라이트': '2023-06-02', '미라시아_얼그레이 하이볼': '2023-01-02',
    '미라시아_유자 하이볼': '2023-03-17', '미라시아_잭 애플 토닉': '2023-09-09',
    '미라시아_칠리 치즈 프라이': '2023-06-03', '미라시아_코카콜라': '2023-06-02',
    '미라시아_코카콜라(제로)': '2023-06-12', '미라시아_콥 샐러드': '2023-12-08',
    '미라시아_파스타면 추가(150g)': '2023-06-03', '미라시아_핑크레몬에이드': '2023-03-17',
    '연회장_Cass Beer': '2023-01-06', '연회장_Conference L1': '2023-01-03',
    '연회장_Conference L2': '2023-01-11', '연회장_Conference L3': '2023-01-05',
    '연회장_Conference M1': '2023-01-06', '연회장_Conference M8': '2023-01-09',
    '연회장_Conference M9': '2023-01-06', '연회장_Convention Hall': '2023-01-03',
    '연회장_Cookie Platter': '2023-01-09', '연회장_Grand Ballroom': '2023-01-06',
    '연회장_OPUS 2': '2023-01-05', '연회장_Regular Coffee': '2023-02-24',
    '연회장_공깃밥': '2023-07-21', '연회장_마라샹궈': '2023-09-08',
    '연회장_매콤 무뼈닭발&계란찜': '2023-01-02', '연회장_삼겹살추가 (200g)': '2023-07-21',
    '연회장_왕갈비치킨': '2023-07-22', '카페테리아_단체식 13000(신)': '2023-04-18',
    '카페테리아_단체식 18000(신)': '2023-04-05', '카페테리아_진사골 설렁탕': '2023-12-06',
    '카페테리아_한상 삼겹구이 정식(2인) 소요시간 약 15~20분': '2023-03-17',
    '화담숲주막_느린마을 막걸리': '2023-03-31', '화담숲주막_단호박 식혜 ': '2023-03-31',
    '화담숲주막_병천순대': '2023-03-31', '화담숲주막_스프라이트': '2023-03-31',
    '화담숲주막_참살이 막걸리': '2023-03-31', '화담숲주막_찹쌀식혜': '2023-03-31',
    '화담숲주막_콜라': '2023-03-31', '화담숲주막_해물파전': '2023-03-31',
    '화담숲카페_메밀미숫가루': '2023-03-31', '화담숲카페_아메리카노 HOT': '2023-03-31',
    '화담숲카페_아메리카노 ICE': '2023-03-31', '화담숲카페_카페라떼 ICE': '2023-03-31',
    '화담숲카페_현미뻥스크림': '2023-03-31'
}
launch_dates = {k: pd.to_datetime(v) for k, v in launch_dates.items()}

def add_future_meta_row(date, key):
    """모델 입력 피처(학습 시 사용한 use_cols 기준)와 일치하도록 미래 1행 메타 피처 생성"""
    row = pd.DataFrame({"영업일자":[pd.Timestamp(date)], "영업장명_메뉴명":[key]})
    d = row.loc[0, "영업일자"]

    # 기본 달력
    row["년"] = d.year
    row["월"] = d.month
    row["일"] = d.day
    row["요일"] = d.dayofweek
    row["주말여부"] = int(row.loc[0,"요일"] in [5,6])

    # 공휴일 & 휴무일
    hol = is_holiday(d)
    row["공휴일여부"] = int(hol)
    row["휴무일여부"] = int(hol or bool(row.loc[0,"주말여부"]))

    # 사이클릭
    row["요일_sin"] = np.sin(2*np.pi*row.loc[0,"요일"]/7.0)
    row["요일_cos"] = np.cos(2*np.pi*row.loc[0,"요일"]/7.0)
    row["월_sin"]   = np.sin(2*np.pi*(row.loc[0,"월"]-1)/12.0)
    row["월_cos"]   = np.cos(2*np.pi*(row.loc[0,"월"]-1)/12.0)

    # 계절
    m = row.loc[0,"월"]
    if m in [12,1,2]:
        season = 0
    elif m in [3,4,5]:
        season = 1
    elif m in [6,7,8]:
        season = 2
    else:
        season = 3
    row["계절(겨울0봄1여름2가을3)"] = np.int8(season)

    # 출시 파생
    row["신규메뉴여부"] = int(key in launch_dates)
    if key in launch_dates and d >= launch_dates[key]:
        row["출시일로부터경과일"] = int((d - launch_dates[key]).days)
    else:
        row["출시일로부터경과일"] = 0

    # 출시 후 주차 & 더미 (학습 시 사용했다면 동일하게)
    row["출시후_주차"] = (row["출시일로부터경과일"] // 7)
    row["출시_0주"]   = (row["출시후_주차"] == 0).astype(int)
    row["출시_1_2주"] = row["출시후_주차"].between(1,2).astype(int)
    row["출시_3_4주"] = row["출시후_주차"].between(3,4).astype(int)
    row["출시_5주이상"] = (row["출시후_주차"] >= 5).astype(int)

    return row

def predict_group_autoreg(g: pd.DataFrame):
    key = g[cat_col].iloc[0]
    g = g.sort_values("영업일자").copy()
    g = g.dropna(subset=["매출수량"])
    assert len(g) >= 28, f"{key}: 28일 히스토리 부족"

    hist_vals = g["매출수량"].values.tolist()[-28:]
    hist_dates = g["영업일자"].tolist()[-28:]
    last_date = g["영업일자"].max()
    preds = []

    for h in range(1, 8):
        cur_date = last_date + pd.Timedelta(days=h)
        row = add_future_meta_row(cur_date, key)

        # lags
        def lag(n): return hist_vals[-n] if len(hist_vals) >= n else np.nan
        row["lag_1"], row["lag_7"], row["lag_14"], row["lag_28"] = lag(1), lag(7), lag(14), lag(28)

        # rolling (과거값만)
        def rmean(n):
            arr = hist_vals[-n:] if len(hist_vals) else []
            return float(np.mean(arr)) if arr else 0.0
        def rsum(n):
            arr = hist_vals[-n:] if len(hist_vals) else []
            return float(np.sum(arr)) if arr else 0.0
        row["roll7_mean"], row["roll7_sum"], row["roll14_mean"], row["roll28_mean"] = \
            rmean(7), rsum(7), rmean(14), rmean(28)

        # 같은 요일 평균(최근 28일 범위)
        cur_dow = cur_date.dayofweek
        hist_dows = [pd.Timestamp(d).dayofweek for d in hist_dates]
        same_idx = [i for i in range(len(hist_vals)) if hist_dows[i] == cur_dow]
        row["same_dow_mean_28"] = float(np.mean([hist_vals[i] for i in same_idx])) if same_idx else 0.0

        # 모델 입력 정렬
        X = row.reindex(columns=use_cols).copy()
        X[cat_col] = X[cat_col].astype("category")
        for c in X.columns:
            if c != cat_col:
                X[c] = X[c].fillna(0)

        yhat = float(model.predict(X, num_iteration=getattr(model, "best_iteration_", None))[0])
        yhat = max(0.0, yhat)  # 음수 방지
        preds.append(yhat)

        # autoreg 업데이트
        hist_vals.append(yhat); hist_dates.append(cur_date)
        if len(hist_vals) > 28:
            hist_vals, hist_dates = hist_vals[-28:], hist_dates[-28:]

    return preds  # 길이 7

# ===== 샘플 제출 파일 불러오기 =====
sub = pd.read_csv(SAMPLE_PATH)
menu_cols = [c for c in sub.columns if c != "영업일자"]
sub[menu_cols] = sub[menu_cols].astype(float)

# ===== TEST 파일 순회 =====
test_files = sorted(glob.glob(os.path.join(TEST_DIR, "TEST_*.csv")))
print("Found:", test_files)

for f in test_files:
    test_name = os.path.splitext(os.path.basename(f))[0]  # TEST_00
    test_id = test_name.split("_")[1]                     # 00
    df = pd.read_csv(f, parse_dates=["영업일자"])

    # 타입/정렬
    if cat_col in df.columns:
        df[cat_col] = df[cat_col].astype("category")
    df = df.sort_values([cat_col, "영업일자"]).copy()

    # 메뉴 단위 예측
    for key, g in df.groupby(cat_col, observed=True):
        preds7 = predict_group_autoreg(g)

        # 제출 파일에 해당 메뉴 열이 없으면 skip
        if key not in menu_cols:
            continue

        # +1~+7일 채우기
        for k in range(1, 8):
            ridx = sub.index[sub["영업일자"] == f"{test_name}+{k}일"]
            if len(ridx) == 1:
                sub.loc[ridx[0], key] = preds7[k-1]

# (선택) 후처리
# sub[menu_cols] = sub[menu_cols].clip(lower=0).round(4)

# ===== 저장 =====
sub.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print("Saved submission ->", OUT_PATH)


Found: ['/content/drive/MyDrive/lg_aimers_2/data/test/TEST_00.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_01.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_02.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_03.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_04.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_05.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_06.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_07.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_08.csv', '/content/drive/MyDrive/lg_aimers_2/data/test/TEST_09.csv']
Saved submission -> /content/drive/MyDrive/lg_aimers_2/submission_lightgbm_05.csv
